In [1]:
!pip install faiss-cpu sentence-transformers langchain pymupdf deep-translator

In [ ]:
import faiss
import pickle
import numpy as np
from pathlib import Path
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

In [ ]:
carpeta_txt = Path("preprocess_pdf")
archivos_txt = [
    carpeta_txt / "file_translated1.txt",
    carpeta_txt / "file_translated2.txt",
    carpeta_txt / "file_translated3.txt",
]

contenido_total = ""
for archivo in archivos_txt:
    contenido_total += archivo.read_text(encoding="utf-8") + "\n"

print(f"Total de caracteres combinados: {len(contenido_total)}")


✅ Total de caracteres combinados: 1113038


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = text_splitter.split_text(contenido_total)

print(f"Número de chunks generados: {len(chunks)}")
chunks[:2]

✅ Número de chunks generados: 2904


['La tuberculosis (TB) es una enfermedad en el aire causada por la bacteria Mycobacterium \nTuberculosis (M. tuberculosis) (Figura 1.1).  M. tuberculosis y siete muy de cerca \nEspecies micobacterianas relacionadas (M. Bovis, M. africanum, M. microti, M. Caprae,  \nM. pinnipedii, M. canetti y M. mungi) comprenden lo que se conoce como el  \nM. Complejo de tuberculosis. La mayoría, pero no todas, de estas especies han sido encontradas',
 'M. Complejo de tuberculosis. La mayoría, pero no todas, de estas especies han sido encontradas \ncausar enfermedad en humanos. En los Estados Unidos, la mayoría de los casos de TB son causados \npor M. tuberculosis.  Los organismos de M. tuberculosis también se llaman bacilos tubérculos.\nFigura 1.1\nMycobacterium \ntuberculosis \nTransmisión de TB\nM. La tuberculosis se transporta en partículas en el aire de 1–5 micras de diámetro llamado']

In [ ]:
model = SentenceTransformer("distiluse-base-multilingual-cased")
embeddings = model.encode(chunks)

print(f"Shape de los embeddings: {embeddings.shape}")

d:\UTMACH\9no Semestre\TESIS\preprocess_pdfs\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\UTMACH\9no Semestre\TESIS\preprocess_pdfs\.venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\camil\.cache\huggingface\hub\models--sentence-transformers--distiluse-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode

✅ Shape de los embeddings: (2904, 512)


In [ ]:
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

faiss.write_index(index, "faiss_index.bin")

with open("faiss_chunks.pkl", "wb") as f:
    pickle.dump(chunks, f)

print("Guardado exitoso de FAISS index y chunks.")

✅ Guardado exitoso de FAISS index y chunks
